<a href="https://colab.research.google.com/github/frnocentini/brain-tumor-object-dectection/blob/main/Brain_Tumor_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%cd '/content/drive/MyDrive/dataset brain tumor/dataset'

/content/drive/MyDrive/dataset brain tumor/dataset


In [10]:
!unzip 'axial_t1wce_2_class' -d '/dataset unzipped'

Archive:  axial_t1wce_2_class.zip
  inflating: /dataset unzipped/axial_t1wce_2_class/axial_t1wce_2_class.yaml  
   creating: /dataset unzipped/axial_t1wce_2_class/images/
   creating: /dataset unzipped/axial_t1wce_2_class/images/test/
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00018_101.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00018_109.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00018_111.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00018_117.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00018_121.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00018_126.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00018_134.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00019_66.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/images/test/00019_75.jpg  
  inflating: /dataset unzipped/axial_t1wce_2_class/image

In [11]:
!unzip 'coronal_t1wce_2_class' -d '/dataset unzipped'

Archive:  coronal_t1wce_2_class.zip
  inflating: /dataset unzipped/coronal_t1wce_2_class/coronal_t1wce_2_class.yaml  
   creating: /dataset unzipped/coronal_t1wce_2_class/images/
   creating: /dataset unzipped/coronal_t1wce_2_class/images/test/
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/13.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/15.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/17.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/19.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/20.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/23.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/33.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/34.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/35.jpg  
  inflating: /dataset unzipped/coronal_t1wce_2_class/images/test/36.jpg  
  inflating: /d

In [12]:
!unzip 'sagittal_t1wce_2_class' -d '/dataset unzipped'

Archive:  sagittal_t1wce_2_class.zip
   creating: /dataset unzipped/sagittal_t1wce_2_class/images/
   creating: /dataset unzipped/sagittal_t1wce_2_class/images/test/
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00000_102.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00000_121.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00000_140.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00000_159.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00002_110.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00002_129.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00002_147.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00002_166.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00003_154.jpg  
  inflating: /dataset unzipped/sagittal_t1wce_2_class/images/test/00003_173.jpg  
  inflating: /

In [48]:
# Data Manipulation
import numpy as np
import pandas as pd
import os
# Visualization/Image Processing
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
# Machine Learning
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, Flatten, MaxPool2D, MaxPooling2D, Dense, Activation, Dropout
from PIL import Image
# Other
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import xml.etree.ElementTree as ET
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import imutils
from PIL import Image, ImageEnhance

In [49]:
%cd '/dataset unzipped'

/dataset unzipped


In [50]:
def crop_brain_contour(image, plot=False):
    
    #import imutils
    #import cv2
    #from matplotlib import pyplot as plt
    
    brightness = 30
    image = np.int16(image)
    image = image + brightness
    image = np.clip(image,0,255)
    image = np.uint8(image)
    #image_enhance = ImageEnhance.Brightness(image)
    #factor = 1.5
    #image = image_enhance.enhance(factor)

    # Convert the image to grayscale, and blur it slightl
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    #thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)[1]
    #thresh = cv2.erode(thresh, None, iterations=2)
    #thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    

    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        
        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        
        plt.title('Original Image')
            
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Cropped Image')
        
        plt.show()

    #add contrast
    #kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    # Top Hat Transform
    #topHat = cv2.morphologyEx(new_image, cv2.MORPH_TOPHAT, kernel)
    # Black Hat Transform
    #blackHat = cv2.morphologyEx(new_image, cv2.MORPH_BLACKHAT, kernel)
    
    #new_image = new_image + topHat - blackHat
    
    return new_image,extLeft,extRight,extTop,extBot

In [51]:
#Codice per rendere l'immagine quadrata 
def make_square(im, fill_color=(0, 0, 0, 0)):
    x, y = im.shape[0], im.shape[1]
    size = max(x, y)
    BLACK = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im,int((size-y)/2),int((size-y)/2),int((size-x)/2),int((size-x)/2),cv2.BORDER_CONSTANT, value = BLACK)
    return new_im

In [52]:
lab = ['axial_t1wce_2_class','coronal_t1wce_2_class', 'sagittal_t1wce_2_class']
X_train = []
bb = []
temp_array  = []
image_size = 180
for i in lab:
    folderPath = os.path.join('./',i,'images','train')
    for j in sorted(tqdm(os.listdir(folderPath))):
        img = cv2.imread(os.path.join(folderPath,j))
        if img.shape[0] != img.shape[1]:
            img = make_square(img, fill_color=(0, 0, 0, 0))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        
y_train = []
for i in lab:
    labelsPath = os.path.join('./',i,'labels','train')
    for j in tqdm(os.listdir(labelsPath)):
        with open(os.path.join(labelsPath,j), encoding="utf-8") as f:
            count_line = 0
            for line in f:
                count_line += 1
                num = list(map(float,line.split(" ")))
                temp_array = []
                for k in num:
                    temp_array.append(k)
                #print(temp_array)
                if count_line == 1:
                    y_train.append(temp_array[0])
                #print(bb_train[image_index]) 


X_test = []
for i in lab:
    folderPath = os.path.join('./',i,'images','test')
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        if img.shape[0] != img.shape[1]:
            img = make_square(img, fill_color=(0, 0, 0, 0))
        img = cv2.resize(img,(image_size, image_size))
        X_test.append(img)

y_test = []
for i in lab:
    labelsPath = os.path.join('./',i,'labels','test')
    for j in tqdm(os.listdir(labelsPath)):
        with open(os.path.join(labelsPath,j), encoding="utf-8") as f:
            count_line = 0
            for line in f:
                count_line += 1
                num = list(map(float,line.split(" ")))
                temp_array = []
                for k in num:
                    temp_array.append(k)
                print(temp_array)
                if count_line ==1:
                    y_test.append(temp_array[0])
                #print(bb_test[image_index])

#print(y_test)



100%|██████████| 75/75 [00:00<00:00, 1959.54it/s]


[0.0, 0.359742, 0.579225, 0.116197, 0.134977]
[0.0, 0.634977, 0.513498, 0.089202, 0.090376]
[0.0, 0.608568, 0.607981, 0.10446, 0.089202]
[0.0, 0.605634, 0.56338, 0.126761, 0.126761]
[0.0, 0.669014, 0.455986, 0.089202, 0.097418]
[0.0, 0.367371, 0.588615, 0.089202, 0.120892]
[0.0, 0.650822, 0.540493, 0.191315, 0.198357]
[0.0, 0.680751, 0.555751, 0.058685, 0.048122]
[0.0, 0.605047, 0.444249, 0.109155, 0.11385]
[0.0, 0.600939, 0.384977, 0.140845, 0.140845]
[0.0, 0.421362, 0.680164, 0.157277, 0.188967]
[0.0, 0.605047, 0.56338, 0.127934, 0.140845]
[0.0, 0.58392, 0.418427, 0.062207, 0.071596]
[0.0, 0.623239, 0.571596, 0.089202, 0.086854]
[0.0, 0.364437, 0.589789, 0.16784, 0.137324]
[0.0, 0.415493, 0.673709, 0.131455, 0.140845]
[0.0, 0.39554, 0.471244, 0.117371, 0.18662]
[0.0, 0.376174, 0.426643, 0.092723, 0.083333]
[0.0, 0.611502, 0.534624, 0.15493, 0.078638]
[0.0, 0.602113, 0.671362, 0.178404, 0.161972]
[0.0, 0.430751, 0.646127, 0.086854, 0.090376]
[0.0, 0.430164, 0.638498, 0.11385, 0.138498

100%|██████████| 78/78 [00:00<00:00, 2245.04it/s]


[0.0, 0.45892, 0.276408, 0.058685, 0.078638]
[1.0, 0.302817, 0.359155, 0.15493, 0.159624]
[0.0, 0.633216, 0.414906, 0.132629, 0.179577]
[1.0, 0.363263, 0.214202, 0.142019, 0.10446]
[1.0, 0.38439, 0.303404, 0.085681, 0.080986]
[0.0, 0.442488, 0.153756, 0.098592, 0.117371]
[0.0, 0.432512, 0.28169, 0.076291, 0.065728]
[1.0, 0.386737, 0.321596, 0.17723, 0.126761]
[1.0, 0.382629, 0.337441, 0.190141, 0.123239]
[1.0, 0.666667, 0.423709, 0.112676, 0.171362]
[0.0, 0.424296, 0.271714, 0.221831, 0.170188]
[1.0, 0.481221, 0.282277, 0.046948, 0.116197]
[1.0, 0.315728, 0.346831, 0.089202, 0.125587]
[0.0, 0.641432, 0.401408, 0.137324, 0.126761]
[1.0, 0.428404, 0.174883, 0.051643, 0.046948]
[1.0, 0.490023, 0.285798, 0.080986, 0.139671]
[0.0, 0.642019, 0.397887, 0.115023, 0.098592]
[1.0, 0.450117, 0.237676, 0.109155, 0.092723]
[0.0, 0.617371, 0.101526, 0.053991, 0.059859]
[1.0, 0.6473, 0.339789, 0.045775, 0.062207]
[1.0, 0.409038, 0.336854, 0.158451, 0.100939]
[1.0, 0.349765, 0.188967, 0.077465, 0.0751

100%|██████████| 70/70 [00:00<00:00, 2461.16it/s]

[1.0, 0.378521, 0.490023, 0.09507, 0.099765]
[0.0, 0.713615, 0.306338, 0.105634, 0.096244]
[0.0, 0.25, 0.40669, 0.239437, 0.127934]
[1.0, 0.425469, 0.464202, 0.142019, 0.092723]
[1.0, 0.471831, 0.298709, 0.107981, 0.142019]
[1.0, 0.503521, 0.258216, 0.098592, 0.103286]
[0.0, 0.243545, 0.319836, 0.106808, 0.09507]
[0.0, 0.242958, 0.419014, 0.269953, 0.150235]
[1.0, 0.328052, 0.372653, 0.052817, 0.059859]
[1.0, 0.404343, 0.426643, 0.109155, 0.118545]
[1.0, 0.526408, 0.366784, 0.078638, 0.090376]
[1.0, 0.407277, 0.453052, 0.115023, 0.098592]
[0.0, 0.607981, 0.321596, 0.119718, 0.117371]
[0.0, 0.708333, 0.357981, 0.10446, 0.105634]
[1.0, 0.284038, 0.424296, 0.185446, 0.153756]
[0.0, 0.463028, 0.339789, 0.156103, 0.120892]
[1.0, 0.61385, 0.407277, 0.08216, 0.08216]
[1.0, 0.485915, 0.471831, 0.215962, 0.122066]
[0.0, 0.463028, 0.347418, 0.200704, 0.169014]
[0.0, 0.243545, 0.264671, 0.09507, 0.125587]
[0.0, 0.195423, 0.339202, 0.090376, 0.061033]
[1.0, 0.410798, 0.459507, 0.152582, 0.123239]


In [28]:
X_train

[array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 

In [53]:
#prepare dataset 
train_dataset = ImageDataGenerator()
val_dataset = ImageDataGenerator()
test_dataset = ImageDataGenerator()

In [54]:
#array conversion
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(y_train)
Y_test = np.array(y_test)

In [55]:
#Shuffle
X_train, y_train = shuffle(X_train, Y_train)

In [56]:
#Validation set created
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, shuffle = False)

In [57]:
BATCH_SIZE = 16
train_dataset = train_dataset.flow(X_train, y_train, batch_size = BATCH_SIZE)
val_dataset = val_dataset.flow(X_val, y_val, batch_size = BATCH_SIZE)

In [58]:
cnn = Sequential()
img_width = image_size
img_height = image_size
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(64, (3, 3), activation="relu", input_shape=(img_width, img_height, 3)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Flatten())
cnn.add(Dense(activation = 'relu', units = 128))
cnn.add(Dense(activation = 'relu', units = 64))
cnn.add(Dense(activation = 'sigmoid', units = 1))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [59]:
cnn.compile(
    loss= 'binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(1e-3),
    metrics= ['accuracy']
)

In [60]:
cnn.fit(
        train_dataset,
        epochs=50,
        validation_data = val_dataset
        )

Epoch 1/50
50/50 [==============================] - 28s 528ms/step - loss: 0.8033 - accuracy: 0.4886 - val_loss: 0.6833 - val_accuracy: 0.5909
Epoch 2/50
50/50 [==============================] - 26s 516ms/step - loss: 0.6894 - accuracy: 0.5316 - val_loss: 0.7039 - val_accuracy: 0.4545
Epoch 3/50
50/50 [==============================] - 26s 515ms/step - loss: 0.6826 - accuracy: 0.5392 - val_loss: 0.7115 - val_accuracy: 0.4205
Epoch 4/50
50/50 [==============================] - 26s 516ms/step - loss: 0.6847 - accuracy: 0.5418 - val_loss: 0.6984 - val_accuracy: 0.5682
Epoch 5/50
50/50 [==============================] - 26s 514ms/step - loss: 0.6769 - accuracy: 0.5620 - val_loss: 0.7074 - val_accuracy: 0.4659
Epoch 6/50
50/50 [==============================] - 26s 515ms/step - loss: 0.6714 - accuracy: 0.5734 - val_loss: 0.7007 - val_accuracy: 0.5682
Epoch 7/50
50/50 [==============================] - 26s 523ms/step - loss: 0.6510 - accuracy: 0.5899 - val_loss: 0.7118 - val_accuracy: 0.4773

KeyboardInterrupt: ignored

In [ ]:
#iou = IoU(y_true, y_pred, 4, [1])
model = tf.keras.models.load_model('./model_checkpoint_SCRATCH/model_SCRATCH.h5',compile = False)

In [ ]:
model.compile(
    loss={
        'coords': 'mae'
    },
    optimizer=tf.keras.optimizers.Adam(1e-3),
    metrics=[IoU]
)

In [ ]:
model.evaluate(test_dataset)